In [2]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go

In [3]:
import plotly.graph_objects as go

labels = ['cancer','none cancer']
values = [89117,130908]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0.06])])

fig.update_layout(autosize=False, width=600, height = 400,
                  
                  title ={'text':'lable distribution','y':0.9,'x':0.5},
                 
                  font = dict(size = 18))
                  
fig.show()

In [4]:
dir_images_train = '/kaggle/input/histopathology-cancer-detection/train'
labels_train = pd.read_csv('/kaggle/input/histopathology-cancer-detection/train_labels.csv')
images_train = labels_train
images_train['label'] = images_train['label'].astype(str)
images_train['id'] = labels_train['id'] + '.tif'

dir_images_test = '/kaggle/input/histopathology-cancer-detection/test'

print(labels_train.head(), '\n')
print(images_train.head(), '\n')
print(labels_train.info())

                                             id label
0  f38a6374c348f90b587e046aac6079959adf3835.tif     0
1  c18f2d887b7ae4f6742ee445113fa1aef383ed77.tif     1
2  755db6279dae599ebb4d39a9123cce439965282d.tif     0
3  bc3f0c64fb968ff4a8bd33af6971ecae77c75e08.tif     0
4  068aba587a4950175d04c680d38943fd488d6a9d.tif     0 

                                             id label
0  f38a6374c348f90b587e046aac6079959adf3835.tif     0
1  c18f2d887b7ae4f6742ee445113fa1aef383ed77.tif     1
2  755db6279dae599ebb4d39a9123cce439965282d.tif     0
3  bc3f0c64fb968ff4a8bd33af6971ecae77c75e08.tif     0
4  068aba587a4950175d04c680d38943fd488d6a9d.tif     0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220025 entries, 0 to 220024
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      220025 non-null  object
 1   label   220025 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB
None


In [7]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, BatchNormalization, Activation
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.optimizers import Adam

In [8]:
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

batch_size = 256

model.build(input_shape=(batch_size, 64, 64, 3))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (256, 62, 62, 16)         448       
_________________________________________________________________
conv2d_1 (Conv2D)            (256, 60, 60, 16)         2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (256, 30, 30, 16)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (256, 28, 28, 32)         4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (256, 26, 26, 32)         9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (256, 13, 13, 32)         0         
_________________________________________________________________
flatten (Flatten)            (256, 5408)               0

In [ ]:
generator = ImageDataGenerator(rescale=1./255, validation_split=0.25)

data_train = generator.flow_from_dataframe(
    dataframe = images_train,
    x_col='id', # filenames
    y_col='label', # labels
    directory=dir_images_train,
    subset='training',
    class_mode='binary',
    batch_size=batch_size,
    target_size=(64, 64))

data_validate=generator.flow_from_dataframe(
    dataframe=images_train,
    x_col='id', # filenames
    y_col='label', # labels
    directory=dir_images_train,
    subset="validation",
    class_mode='binary',
    batch_size=batch_size,
    target_size=(64, 64))

In [ ]:
opt = Adam(learning_rate=0.0001)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit(data_train, validation_data=data_validate, epochs=10)

In [ ]:
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])

plt.title("Model Evaluation")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","Loss","Validation Loss"])

plt.show()